# Chord voicings

I will speak of chords in a way that is much less nuanced than how they're typically spoken of in music theory.

For our purposes, there are two ideas that fall under the category of "chords".

The first are what can be thought of as _classes_ of chords, which are represented by pitch class sets, and are essentially equivalent to what I typically call "scales".

The second are what can be thought of as chord _voicings_, which are represented by pitch sets. 

You can have a chord like Cmaj7, represented by the pitch class set `{0, 4, 7, 11} mod 12 root 0`.

Think of this as an entire class of chords. From it, we can build many specific voicings by constructing pitch sets from this pitch class set.

## A notation for writing out voicings

I've come to find that it's easiest to write out voicings by permuting the pitch classes, and then translating them to pitches.

For example: `[0, 4, 11, 7]` is a permutation of pitch classes (mod 12) which represents the pitch set `{0, 4, 11, 19}`.

This simplifies the algorithm for enumerating chord voicings, because it simply becomes a matter of enumerating permutations of pitch classes.

In order to translate these into pitch sets, we'll need a function that adds an appropriate number of octaves to each pitch class.

In [ ]:
from harmonica.pitch import PitchSet


def pitch_class_permutation_to_pitch_set(pc_perm: list[int], modulus: int) -> PitchSet:
    assert all(
        [0 <= pitch_class < modulus for pitch_class in pc_perm]
    ), "Pitch classes must be non-negative and less than modulus."
    assert modulus > 0, "Modulus must be positive."

    pitches = [pc_perm[0]]

    octave = 0

    for i, pitch_class in enumerate(pc_perm[1:]):
        if pitch_class <= pc_perm[i]:
            octave += 1
        pitches.append(pitch_class + (modulus * octave))

    return PitchSet(pitches)


voicing = pitch_class_permutation_to_pitch_set([4, 11, 7, 4, 2], 12)
print(voicing)  # {4, 11, 19, 28, 38}

PitchSet(pitches=[4, 11, 19, 28, 38])


## Enumerating all voicings of a pitch class set with no repetitions

In [ ]:
from itertools import permutations
from harmonica.pitch import PitchClassSet, PitchSetSeq
from harmonica.utility import GM


def enumerate_voicings_no_repetitions(pitch_class_set: PitchClassSet) -> list[PitchSet]:
    pitch_class_permutations = [
        list(perm) for perm in permutations(pitch_class_set.pitch_classes)
    ]

    pitch_sets = [
        pitch_class_permutation_to_pitch_set(pc_perm, pitch_class_set.modulus)
        for pc_perm in pitch_class_permutations
    ]

    return pitch_sets


pitch_class_set = PitchClassSet([0, 4, 7, 11], modulus=12)
voicings = enumerate_voicings_no_repetitions(pitch_class_set)

for voicing in voicings:
    print(voicing)

# PitchSetSeq(voicings).preview(bass=48, program=GM.ElectricPiano1)

PitchSet(pitches=[0, 4, 7, 11])
PitchSet(pitches=[0, 4, 11, 19])
PitchSet(pitches=[0, 7, 16, 23])
PitchSet(pitches=[0, 7, 11, 16])
PitchSet(pitches=[0, 11, 16, 19])
PitchSet(pitches=[0, 11, 19, 28])
PitchSet(pitches=[4, 12, 19, 23])
PitchSet(pitches=[4, 12, 23, 31])
PitchSet(pitches=[4, 7, 12, 23])
PitchSet(pitches=[4, 7, 11, 12])
PitchSet(pitches=[4, 11, 12, 19])
PitchSet(pitches=[4, 11, 19, 24])
PitchSet(pitches=[7, 12, 16, 23])
PitchSet(pitches=[7, 12, 23, 28])
PitchSet(pitches=[7, 16, 24, 35])
PitchSet(pitches=[7, 16, 23, 24])
PitchSet(pitches=[7, 11, 12, 16])
PitchSet(pitches=[7, 11, 16, 24])
PitchSet(pitches=[11, 12, 16, 19])
PitchSet(pitches=[11, 12, 19, 28])
PitchSet(pitches=[11, 16, 24, 31])
PitchSet(pitches=[11, 16, 19, 24])
PitchSet(pitches=[11, 19, 24, 28])
PitchSet(pitches=[11, 19, 28, 36])


## Including voicings with repeated pitch classes

Obviously, this enumeration doesn't account for all possible ways to voice a pitch class set. There are many voicings in which a pitch class occurs multiple times. 

On a purely mathematical level (which is the level I enjoy working on), there are infinitely many occurrences of a pitch class in the pitch domain. 
Therefore, in order to have a finite enumeration, we must specify some maximum number of repetitions as a parameter.